In [1]:
import sys
sys.path.append('../../Common/')

import CommonBinance

In [2]:
def detectSignal(symbol, from_date, to_date, timeframe):

    import pandas as pd
    import plotly.graph_objects as go
    import redis
    import numpy as np
    from datetime import datetime

    # ##############################################Step 1: Lấy dữ liệu##############################################
    data = CommonBinance.CommonBinance.loaddataBinance_FromTo(symbol, from_date, to_date, timeframe)
    
    # ##############################################Step 2: Chiến lược##############################################  
    # Tính toán các chỉ báo kỹ thuật
    data['HA_Close'] = (data['Open'] + data['High'] + data['Low'] + data['Close']) / 4
    ha_open = [(data['Open'][0] + data['Close'][0]) / 2]  # Khởi tạo giá trị đầu tiên cho Open

    # Tính giá Open của các bar Heikin-Ashi tiếp theo
    for i in range(1, len(data)):
        ha_open.append((ha_open[i-1] + data['HA_Close'][i-1]) / 2)
        
    data['HA_Open'] = ha_open
    data['HA_High'] = data[['High', 'HA_Open', 'HA_Close']].max(axis=1)
    data['HA_Low'] = data[['Low', 'HA_Open', 'HA_Close']].min(axis=1)

    # Tính toán EMA từ giá đóng cửa Heikin-Ashi
    data['SMA'] = data['HA_Close'].rolling(window=20).mean()
    # EMA ngắn hạn
    data['short_ema'] = data['HA_Close'].ewm(span=12, adjust=False).mean()
    # EMA dài hạn
    data['long_ema'] = data['HA_Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = data['short_ema'] - data['long_ema']
    data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Tạo cột tín hiệu mua/bán
    data['Buy_Signal'] = (data['MACD'] > data['Signal_Line']) & (data['Close'] > data['SMA'])
    data['Sell_Signal'] = (data['MACD'] < data['Signal_Line']) & (data['Close'] < data['SMA'])   
    
    # ##############################################Step 3: Đẩy dữ liệu qua Redis##############################################
    # Nếu có tín hiệu thì đẩy qua Redis
    # Datetime  Open    High    Low	Close   Volume  SMA short_ema   long_ema    MACD    Signal_Line Buy_Signal  Sell_Signal
    # Tạo kết nối Redis
    r = redis.Redis(host='localhost', port=6379, db=0)
    # Tạo hash key
    hash_key = 'OG_CRTO_MACD dua tren HA'
    last_record = data.iloc[-1]
   
    # Chuyển đổi record cuối cùng thành từ điển và lưu vào Redis dưới dạng hash
    if(last_record['Buy_Signal'] == True or last_record['Sell_Signal'] == True):
        for field, value in last_record.to_dict().items():
            # Chuyển đổi giá trị uint64 và Timestamp thành chuỗi
            if isinstance(value, pd.Timestamp):
                value = value.isoformat()
            elif isinstance(value, (int, np.uint64)):
                value = str(value)
            r.hset(hash_key, field, value)  
            r.hset(hash_key, 'Symbol', symbol)
            r.hset(hash_key, 'Insertdate', datetime.now().isoformat())
        print(last_record)   
    else:
        print('Không có tín hiệu!')

    # ##############################################Step 4: Vẽ biểu đồ##############################################  
    drawHA(data)
    drawClose(data)
    drawMACD(data)

In [3]:
def drawHA(data):
    import pandas as pd
    import plotly.graph_objects as go

    # Giả sử 'data' là DataFrame chứa dữ liệu giá với các cột 'Open', 'High', 'Low', và 'Close'
    # Và bạn đã tính toán 'HA_Close', 'HA_Open', 'HA_High', và 'HA_Low' như trong đoạn code trước

    # Tạo biểu đồ nến Heikin-Ashi với plotly
    fig = go.Figure(data=[go.Candlestick(x=data['Datetime'],  # Sử dụng cột 'Datetime'
                                        open=data['HA_Open'],
                                        high=data['HA_High'],
                                        low=data['HA_Low'],
                                        close=data['HA_Close'],
                                        increasing_line_color='green',
                                        decreasing_line_color='red')])

    # Cập nhật layout của biểu đồ
    fig.update_layout(title='Heikin-Ashi Chart',
                    xaxis_title='Date',
                    yaxis_title='Price',
                    xaxis_rangeslider_visible=True)

    # Hiển thị biểu đồ
    fig.show()

In [4]:
def drawClose(data):
    import pandas as pd
    import plotly.graph_objects as go

    # Giả sử 'data' là DataFrame chứa dữ liệu giá với các cột 'Open', 'High', 'Low', và 'Close'
    # Và bạn đã tính toán 'HA_Close', 'HA_Open', 'HA_High', và 'HA_Low' như trong đoạn code trước

    # Tạo biểu đồ nến Heikin-Ashi với plotly
    fig = go.Figure(data=[go.Candlestick(x=data['Datetime'],  # Sử dụng cột 'Datetime'
                                        open=data['Open'],
                                        high=data['High'],
                                        low=data['Low'],
                                        close=data['Close'],
                                        increasing_line_color='green',
                                        decreasing_line_color='red')])

    # Cập nhật layout của biểu đồ
    fig.update_layout(title='OHLC Chart',
                    xaxis_title='Date',
                    yaxis_title='Price',
                    xaxis_rangeslider_visible=True)

    # Hiển thị biểu đồ
    fig.show()

In [5]:
def drawMACD(data):
    import plotly.graph_objects as go

    # Giả sử bạn đã có DataFrame 'data' với các cột 'MACD' và 'Signal_Line' đã được tính toán

    # Tạo figure với plotly
    fig = go.Figure()

    # Thêm đường MACD
    fig.add_trace(go.Scatter(x=data['Datetime'], y=data['MACD'], mode='lines', name='MACD', line=dict(color='blue')))

    # Thêm đường tín hiệu
    fig.add_trace(go.Scatter(x=data['Datetime'], y=data['Signal_Line'], mode='lines', name='Signal Line', line=dict(color='red')))

    # Cập nhật layout
    fig.update_layout(
        title='MACD on Heikin-Ashi Closing Prices',
        xaxis_title='Date',
        yaxis_title='MACD Value',
        legend_title='Indicator',
        xaxis_rangeslider_visible=True
    )

    # Hiển thị biểu đồ
    fig.show()

In [6]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import ta
import schedule
import time

def schedule_detectSignal(timeframe):
    symbol = 'NEARUSDT'
    from_date = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
    to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    detectSignal(symbol, from_date, to_date, timeframe)

# Danh sách các phút cụ thể bạn muốn hàm được chạy
run_minutes = list(range(3,60,30))
while True:
    timeframe = '30m'
    current_time = datetime.now()
    current_minute = current_time.minute
    # Kiểm tra xem phút hiện tại có nằm trong danh sách các phút cần chạy hàm không
    if current_minute in run_minutes:
        # Kiểm tra xem đã chạy hàm trong phút hiện tại hay chưa
        last_run = getattr(schedule_detectSignal, 'last_run', None)
        if last_run is None or last_run != current_minute:
            schedule_detectSignal(timeframe)
            # Lưu lại phút cuối cùng mà hàm đã chạy
            setattr(schedule_detectSignal, 'last_run', current_minute)

    # Nghỉ 1 giây trước khi kiểm tra lại
    time.sleep(1)

Datetime       2024-02-27 11:00:00
Open                         3.995
High                         3.995
Low                          3.981
Close                        3.982
Volume                     28810.7
HA_Close                   3.98825
HA_Open                   3.990347
HA_High                      3.995
HA_Low                       3.981
SMA                       4.023112
short_ema                 4.006859
long_ema                  4.020721
MACD                     -0.013862
Signal_Line              -0.006648
Buy_Signal                   False
Sell_Signal                   True
Name: 118, dtype: object
